In [4]:
# For developers who want to use the latest development version or the library locally
# Use poetry to install dependencies
import sys
sys.path.append("../") # Or change to the folder to the direction of 

%load_ext autoreload
%autoreload 2

import bechdelaidemo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bechdel test on a dialogue scene
The test is already done on a dialogue scene. So easier, because we don't have to extract dialogue segments from a video

## Test on 2 dialogue scenes

In [1]:
path1 = "https://www.youtube.com/watch?v=b2f2Kqt_KcE&ab_channel=Movieclips" #Devils wears prada
path2 = "https://www.youtube.com/watch?v=FDFdroN7d0w" #Marriage story

## Download videos and audios from Youtube
Inspiration from https://huggingface.co/spaces/vumichien/whisper-speaker-diarization/blob/main/app.py

In [5]:
from bechdelaidemo.utils import download_youtube_video

In [6]:
download_youtube_video(path2,"video.mp4")

Task Completed!


In [7]:
video_path = "video.mp4"

## Convert video to audio wav file

In [8]:
from bechdelaidemo.utils import extract_audio_from_movie

In [9]:
extract_audio_from_movie(video_path,".wav")

MoviePy - Writing audio in video.wav


MoviePy - Done.


In [10]:
audio_file = "video.wav"

## Clean audio
- https://huggingface.co/speechbrain/sepformer-wham16k-enhancement
- Spleeter

## Extract info using Whisper

In [11]:
import whisper

In [12]:
model = whisper.load_model("tiny.en")

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 90.2MiB/s]


In [15]:
## Transcribe audio
options = dict(language="en", beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

print(transcribe_options)

result = model.transcribe("video.wav", **transcribe_options)
segments = result["segments"]

{'task': 'transcribe', 'language': 'en', 'beam_size': 5, 'best_of': 5}


In [16]:
result["text"]

" You're being so much like your father. Do not compare me to my father. I didn't compare you, Dan. I said you were acting like him. You're exactly like your mother. Everything you're complaining about her you're doing. You're suffocating in me. First of all, I love my mother. She was a wonderful mother. Just repeating what you told me. Secondly, how dare you compare my mother into my mother? I'm maybe like my father, but I am not like my mother. You are! And you're like my father. You're also like my mother. You're all the bad things about all of these people. But mostly your mother. When we would lie in bed together, sometimes I would look at you and see her and just feel so gross. I felt appalled when you touched her. You're a slob. I mean all the beds, clothes on the camera, and it steps together with you like anything. You want to heal my skin off. You'll never be happy. And now lay her anywhere. You'll think you found some better opposite guy than me. And in a few years, you rebe

In [17]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.0,
  'text': " You're being so much like your father.",
  'tokens': [921, 821, 852, 523, 881, 588, 534, 2988, 13],
  'temperature': 0.0,
  'avg_logprob': -0.15723278113370817,
  'compression_ratio': 1.9415807560137457,
  'no_speech_prob': 0.16063228249549866},
 {'id': 1,
  'seek': 0,
  'start': 3.0,
  'end': 5.0,
  'text': ' Do not compare me to my father.',
  'tokens': [2141, 407, 8996, 502, 284, 616, 2988, 13],
  'temperature': 0.0,
  'avg_logprob': -0.15723278113370817,
  'compression_ratio': 1.9415807560137457,
  'no_speech_prob': 0.16063228249549866},
 {'id': 2,
  'seek': 0,
  'start': 5.0,
  'end': 6.0,
  'text': " I didn't compare you, Dan.",
  'tokens': [314, 1422, 470, 8996, 345, 11, 6035, 13],
  'temperature': 0.0,
  'avg_logprob': -0.15723278113370817,
  'compression_ratio': 1.9415807560137457,
  'no_speech_prob': 0.16063228249549866},
 {'id': 3,
  'seek': 0,
  'start': 6.0,
  'end': 7.0,
  'text': ' I said you were acting l

## Know duration

In [18]:
import wave
import contextlib

In [19]:
# Get duration
with contextlib.closing(wave.open(audio_file,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
print(f"conversion to wav ready, duration of audio file: {duration}")

conversion to wav ready, duration of audio file: 258.93


## Speaker diarization

In [1]:
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
from tqdm.auto import tqdm
import numpy as np
import torch

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device="cpu")

OSError: [WinError 1314] Le client ne dispose pas d’un privilège nécessaire: 'C:\\Users\\theo.alvesdacosta\\.cache\\huggingface\\hub\\models--speechbrain--spkrec-ecapa-voxceleb\\snapshots\\5c0be3875fda05e81f3c004ed8c7c06be308de1e\\hyperparams.yaml' -> 'C:\\Users\\theo.alvesdacosta\\.cache\\torch\\pyannote\\speechbrain\\hyperparams.yaml'

In [25]:
def segment_embedding(segment):
    audio = Audio()
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(audio_file, clip)
    return embedding_model(waveform[None])

In [26]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(tqdm(segments)):
    embeddings[i] = segment_embedding(segment)
embeddings = np.nan_to_num(embeddings)
print(f'Embedding shape: {embeddings.shape}')

  0%|          | 0/101 [00:00<?, ?it/s]

NameError: name 'embedding_model' is not defined